# Understand Brazilian E-commerce co.



In [26]:
# Import all of the libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
import matplotlib as mpl
mpl.rcParams['patch.force_edgecolor'] = True
%matplotlib inline


ImportError: cannot import name 'factorial' from 'scipy.misc' (/anaconda3/lib/python3.7/site-packages/scipy/misc/__init__.py)